### 참고 링크
* [참고 영상: Backtesting.py SMA Crossover Strategy Example with Bitcoin: Using Python to Optimize Parameters](https://youtu.be/gdrSo1Yclys)  
* [PyKrx](https://github.com/sharebook-kr/pykrx)

In [560]:
## C:/Users/present-jdnm/anaconda3/Scripts/activate
## %reset -f

# 작업 공간 다운로드

In [561]:
# current_file_name = 'fibonacci'

# import dill
# import os
# # and to load the session again:
# try:
#     dill.load_session(os.getcwd() +'/'+current_file_name + '.pkl')
# except:
#     print('Please check if the pkl file exists')

# 기본 패키지, 모듈 삽입

In [562]:
from pykrx import stock
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
import copy
from tqdm import tqdm

pd.set_option('display.max_columns',None)

## 코스닥 종목 정보 및 데이터 불러오기

In [563]:
"""
종목명:종목코드
에스엠:041510
펄어비스:263750
JYP Ent.:035900
삼성전자:005930
"""

market = "KOSPI" #"KOSDAQ"#
duration = ["20000101", "20211231"]
# duration = ["20010101", "20211231"]
duration[0] = (pd.Timestamp(duration[0]) - pd.Timedelta(days=21)).strftime('%Y-%m-%d')
duration[1] = (datetime.datetime.today() + pd.Timedelta(days=1)).strftime('%Y-%m-%d')#- pd.Timedelta(days=3)).strftime('%Y-%m-%d')

# # 종목리스트 추출
# kos_set0 = set(stock.get_market_ticker_list(duration[0], market=market))
# kos_set1 = set(stock.get_market_ticker_list(duration[1], market=market))
# kos_list = list(kos_set0 & kos_set1)


# df_tmp = stock.get_market_cap(duration[1])
# # target_number = '041510'# 에스엠
# # target_number = '263750'# 펄어비스
# # target_number = '035900'# JYP Ent.
# target_number = '005930'# 삼성전자
# target_number = '006400'# 삼성SDI
# target_number = '123310'# 타이거inverse
# target_number = '139480'# 이마트
# target_number = '009150'# 삼성전기
# target_number = '069960'# 현대백화점
# target_number = '017800'# 현대엘리베이터
# target_number = '119860'# 다나와
# target_number = '336060'# 웨이버스
# target_number = '089590'# 제주항공
# target_number = '051980'# 중앙디앤엠
target_number = '003010'# 혜인


# # kos_list = [target_number]
# print(f'종목코드:{target_number}\n',df_tmp.loc[target_number])

# # kosdaq = stock.get_index_ohlcv(duration[0], duration[1], "2001")
# # kosdaq.drop('거래대금',axis=1,inplace=True)
# # kosdaq.columns = ['Open', 'High', 'Low', 'Close','Volume']
# # kosdaq.head(5)

In [564]:
df = stock.get_market_ohlcv(duration[0], duration[1], target_number)
df.columns = ['Open', 'High', 'Low', 'Close','Volume']
df

,Open,High,Low,Close,Volume
날짜,,,,,
1999-12-13,2700,2700,2500,2580,181810
1999-12-14,2510,2660,2510,2600,248430
1999-12-15,2610,2610,2410,2410,124330
1999-12-16,2540,2580,2410,2440,174280
1999-12-17,2510,2550,2300,2350,148800
...,...,...,...,...,...
2022-06-02,10350,10600,10250,10600,2873220
2022-06-03,11900,12400,10800,11150,24528040
2022-06-07,11200,11750,10800,11550,7400737


In [565]:

## 증감으로 환산
df_percentage = df.diff()
for col in ['Open',	'High',	'Low',	'Close']:
    df_percentage[col] = (df_percentage[col] / df['Close'].shift(1))# *100
df_percentage['Volume'] = df_percentage['Volume'] /df['Volume'].shift(1)


def sign(array):
    array = np.round(np.sign(np.sign(np.sign(array)-0.5)+1))
    return array


def sign_2nd(array):
    array=np.round(np.sign(array))
    return array

for day in range(1,10+1):
    df_percentage['Close '+'{0:02d}'.format(day)+ 'days before'] = df_percentage['Close'].shift(day)
    # df_percentage['Open '+'{0:02d}'.format(day)+ 'days before'] = df_percentage['Open'].shift(day)
    # df_percentage['High '+'{0:02d}'.format(day)+ 'days before'] = df_percentage['High'].shift(day)
    # df_percentage['Low '+'{0:02d}'.format(day)+ 'days before'] = df_percentage['Low'].shift(day)
    # df_percentage['Volume '+'{0:02d}'.format(day)+ 'days before'] = df_percentage['Volume'].shift(day)
    
df_percentage['Close-Open'] = df_percentage['Close'] - df_percentage['Open']

# value_caculation_duration = 1
# for day in range(1,value_caculation_duration+1):
#     df_percentage['Close-Open '+'{0:02d}'.format(day)+ 'days after'] = df_percentage['Close-Open'].shift(-day)
    
# df_percentage = df_percentage.drop('Close-Open',axis=1)
# # df_percentage = df_percentage.dropna(axis=0)


# df_percentage['expected_value'] = np.NaN
# for i, col in enumerate(df_percentage.columns[-value_caculation_duration-1:-1]):
#     df_percentage[col] = df_percentage[col]*(0.5**i)
    
# df_percentage['expected_value'] = df_percentage[df_percentage.columns[-value_caculation_duration-1:-1]].mean(axis=1)


df_percentage['expected_value'] = sign(df_percentage['Close-Open'].shift(-1))
# df_percentage = df_percentage.drop(df_percentage.columns[-value_caculation_duration-1:-1],axis=1)

# 전날이 평일인지 휴일인지 & 다음날이 평일인지 휴일인지 추가
# 평일 0, 휴일 1
dow =[]
df_percentage['dow_1days_before'] = df_percentage.index
df_percentage['dow_1days_after'] = df_percentage.index
df_percentage['dow_1days_before'] = (df_percentage['dow_1days_before'] - df_percentage['dow_1days_before'].shift(1)).apply(lambda x : x.days)
df_percentage['dow_1days_after'] = (df_percentage['dow_1days_after'].shift(-1) - df_percentage['dow_1days_after']).apply(lambda x : x.days)
df_percentage['dow_1days_before'] = np.sign(np.sign(df_percentage['dow_1days_before'] -1.5) +1)
df_percentage['dow_1days_after'] = np.sign(np.sign(df_percentage['dow_1days_after'] -1.5) +1)
df_percentage['dow_1days_after'].iloc[-1] = 1

print('shape of data = ',df_percentage.shape)
df_percentage.tail(10)

shape of data =  (5550, 19)


,Open,High,Low,Close,Volume,Close 01days before,Close 02days before,Close 03days before,Close 04days before,Close 05days before,Close 06days before,Close 07days before,Close 08days before,Close 09days before,Close 10days before,Close-Open,expected_value,dow_1days_before,dow_1days_after
날짜,,,,,,,,,,,,,,,,,,,
2022-05-25,-0.075452,-0.204038,-0.035069,-0.002125,-0.747043,-0.036847,-0.027861,0.025510,0.299735,0.133835,0.055556,-0.020218,-0.003101,0.006240,-0.036090,0.073326,0.0,0.0,0.0
2022-05-26,0.047923,0.076677,0.019169,-0.025559,0.758967,-0.002125,-0.036847,-0.027861,0.025510,0.299735,0.133835,0.055556,-0.020218,-0.003101,0.006240,-0.073482,1.0,0.0,0.0
2022-05-27,-0.079781,-0.078689,-0.014208,0.016393,-0.459983,-0.025559,-0.002125,-0.036847,-0.027861,0.025510,0.299735,0.133835,0.055556,-0.020218,-0.003101,0.096175,1.0,0.0,1.0
2022-05-30,0.022581,0.077419,0.026882,0.091398,1.866114,0.016393,-0.025559,-0.002125,-0.036847,-0.027861,0.025510,0.299735,0.133835,0.055556,-0.020218,0.068817,0.0,1.0,0.0
2022-05-31,0.059113,0.064039,0.054187,0.039409,-0.228558,0.091398,0.016393,-0.025559,-0.002125,-0.036847,-0.027861,0.025510,0.299735,0.133835,0.055556,-0.019704,0.0,0.0,1.0
2022-06-02,0.040758,-0.028436,0.044550,0.004739,-0.689694,0.039409,0.091398,0.016393,-0.025559,-0.002125,-0.036847,-0.027861,0.025510,0.299735,0.133835,-0.036019,0.0,1.0,0.0
2022-06-03,0.146226,0.169811,0.051887,0.051887,7.536778,0.004739,0.039409,0.091398,0.016393,-0.025559,-0.002125,-0.036847,-0.027861,0.025510,0.299735,-0.094340,1.0,0.0,1.0
2022-06-07,-0.062780,-0.058296,0.000000,0.035874,-0.698274,0.051887,0.004739,0.039409,0.091398,0.016393,-0.025559,-0.002125,-0.036847,-0.027861,0.025510,0.098655,0.0,1.0,0.0
2022-06-08,0.017316,-0.025974,-0.034632,-0.090909,-0.585257,0.035874,0.051887,0.004739,0.039409,0.091398,0.016393,-0.025559,-0.002125,-0.036847,-0.027861,-0.108225,1.0,0.0,0.0


## 인공지능 모델 구동

In [566]:
import lightgbm
print('lightgbm version',lightgbm.__version__)
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings("ignore")

lightgbm version 3.2.1


In [567]:
X = df_percentage.drop('expected_value',axis=1)
Y = df_percentage['expected_value']
split_prob=[0.6,0.2,0.1]
# X_train = X.iloc[round(len(X)*0.20):-1]
# Y_train = Y.iloc[round(len(Y)*0.20):-1]
X_train = X.iloc[0:round(len(X)*0.60)]
Y_train = Y.iloc[0:round(len(Y)*0.60)]
X_validation = X.iloc[round(len(X)*0.60):round(len(X)*0.80)]
Y_validation = Y.iloc[round(len(Y)*0.60):round(len(Y)*0.80)]
X_test = X.iloc[round(len(X)*0.80):]
Y_test = Y.iloc[round(len(Y)*0.80):]
# X_validation = X.iloc[0:round(len(X)*0.20)]
# Y_validation = Y.iloc[0:round(len(Y)*0.20)]
# X_test = X.iloc[-1:]
# Y_test = Y.iloc[-1:]

In [568]:
# 앞서 XGBoost와 동일하게 n_estimators는 400 설정. 
lgbm_wrapper = LGBMClassifier(n_estimators=400)

# LightGBM도 XGBoost와 동일하게 조기 중단 수행 가능. 
evals = [(X_validation, Y_validation)]
lgbm_wrapper.fit(X_train, Y_train, early_stopping_rounds=1000, eval_metric="logloss", 
                 eval_set=evals, verbose=True)
preds = lgbm_wrapper.predict(X_test)
pred_proba = lgbm_wrapper.predict_proba(X_test)[:, 1]


[1]	valid_0's binary_logloss: 0.672373
Training until validation scores don't improve for 1000 rounds
[2]	valid_0's binary_logloss: 0.653223
[3]	valid_0's binary_logloss: 0.639544
[4]	valid_0's binary_logloss: 0.628306
[5]	valid_0's binary_logloss: 0.618672
[6]	valid_0's binary_logloss: 0.609333
[7]	valid_0's binary_logloss: 0.603631
[8]	valid_0's binary_logloss: 0.596374
[9]	valid_0's binary_logloss: 0.590935
[10]	valid_0's binary_logloss: 0.586949
[11]	valid_0's binary_logloss: 0.583291
[12]	valid_0's binary_logloss: 0.580272
[13]	valid_0's binary_logloss: 0.57727
[14]	valid_0's binary_logloss: 0.57405
[15]	valid_0's binary_logloss: 0.573569
[16]	valid_0's binary_logloss: 0.571415
[17]	valid_0's binary_logloss: 0.570308
[18]	valid_0's binary_logloss: 0.568996
[19]	valid_0's binary_logloss: 0.568026
[20]	valid_0's binary_logloss: 0.567158
[21]	valid_0's binary_logloss: 0.567099
[22]	valid_0's binary_logloss: 0.567089
[23]	valid_0's binary_logloss: 0.566337
[24]	valid_0's binary_loglos

In [569]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred,average='weighted')
    recall = recall_score(y_test , pred,average='weighted')
    f1 = f1_score(y_test,pred,average='weighted')
    # ROC-AUC 추가 
    #roc_auc = roc_auc_score(y_test, pred_proba,average='weighted')
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, f1))

    return accuracy

In [570]:
score=get_clf_eval(Y_test.iloc[:-1], preds[:-1], pred_proba[:-1])

오차 행렬
[[419 151]
 [146 393]]
정확도: 0.7322, 정밀도: 0.7323, 재현율: 0.7322,    F1: 0.7322, AUC:0.7322


In [571]:
sum(preds)

545.0

In [572]:
# pred_proba_df = df_percentage.iloc[round(len(Y)*0.80):].copy()
# pred_proba_df['expected_value'].iloc[-1] = pred_proba

In [573]:
# pred_proba_df[['Open',	'High',	'Low',	'Close',	'Volume', 'expected_value']].tail(10)